In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from datetime import datetime
import pickle 
import matplotlib.pyplot as plt

def parse_length_of_stay(response):
    number_strings = [str(i) for i in range(10)]

    if isinstance(response, list):
        return np.array([parse_length_of_stay(r) for r in response])
    
    if len(response) == 0:
        return -1
    
    if response[0] not in number_strings:
        return -1
    else: 
        
        for s in range(len(response)):
            if response[s] not in number_strings:
                break
        
        return float(response[s])
    
    
def parse_in_hospital_death(response):
    if isinstance(response, list):
        return np.array([parse_in_hospital_death(r) for r in response])
    
    if len(response) == 0:
        return -1
    
    response = response.lower()
    if response[:2] == 'no':
        return 0
    elif response[:3] == 'yes':
        return 1
    
    return -1


def parse_decompensation(response):
    if isinstance(response, list):
        return np.array([parse_decompensation(r) for r in response])
    
    if len(response) == 0:
        return -1
    
    response = response.lower()
    if response[:2] == 'no':
        return 0
    elif response[:3] == 'yes':
        return 1
    
    return -1

stay_dicts = pickle.load(open('/mnt/c/Users/hhelm/Documents/Helivan/Data/stay_dicts.p', 'rb'))
responses_dict = pickle.load(open('/mnt/c/Users/hhelm/Documents/Helivan/Microsoft/data/multi-task-results.p', 'rb'))

task_list = list(responses_dict.keys())
n_hours_list = list(responses_dict[tasks[0]].keys())
stay_id_list = list(responses_dict[tasks[0]][n_hours_list[0]].keys())

In [7]:
valid_response_dict = {}
result_dict = {}
truth_dict = {}
del_list_by_task = {}

for task in task_list:
    del_list_by_task[task] = []
    valid_response_dict[task] = {}
    result_dict[task] = {}
    truth_dict[task] = {}
    for n_hours in n_hours_list:
        valid_response_dict[task][n_hours]={}
        result_dict[task][n_hours] = {}
        truth_dict[task][n_hours] = {}
        for stay_id in stay_id_list:
            if task == 'length_of_stay':
                temp_array = parse_length_of_stay(responses_dict[task][n_hours][stay_id])
                temp_truth=st
            elif task == 'decompensation':
                temp_array = parse_decompensation(responses_dict[task][n_hours][stay_id])
            else:
                temp_array = parse_in_hospital_death(responses_dict[task][n_hours][stay_id])
                            
            valid_only = temp_array[temp_array != -1]
            
            if len(valid_only) == 0:
                del_list_by_task[task].append(stay_id)
                continue
                
            valid_response_dict[task][n_hours][stay_id] = temp_array
            result_dict[task][n_hours][stay_id] = np.mean(temp_array)
            
            

In [52]:
#- Analyze length of stay prediction
task ='length_of_stay'

los_mean = {}
los_var = {}
del_list = []

for n_hours in n_hours_list:
    los_mean[n_hours] = {}
    los_var[n_hours]={}
    for stay_id in stay_dicts.keys():
        
        temp_array = parse_length_of_stay(responses_dict[task][n_hours][stay_id])
        floats_only = temp_array[temp_array != -1]
        
        if len(floats_only) == 0:
            los_mean[n_hours][stay_id] = -1
            del_list.append(stay_id)
            continue
        
        los_mean[n_hours][stay_id] = np.mean(floats_only)
        los_var[n_hours][stay_id]=np.var(floats_only)
        
abs_errors = {}
variance = {}
for stay_id in stay_dicts.keys():
    if stay_id in del_list:
        continue
        
    los = stay_dicts[stay_id]['los']
    abs_errors[stay_id] = []
    
    for n_hours in n_hours_list:
        abs_errors[stay_id].append(abs(los_results[n_hours][stay_id] -los))
        
    stay_id_ = stay_id
    
avg_across_stays = np.mean(np.array(list(abs_errors.values())), axis=0)